In [60]:
#https://www.youtube.com/watch?v=SdQaSM_DScU&ab_channel=NASAVideo
#https://www.rdocumentation.org/packages/nasapower/versions/3.0.1/topics/parameters
#needed to make web requests
import requests

#store the data we get as a dataframe
import pandas as pd

#convert the response as a strcuctured json
import json

#mathematical operations on lists
import numpy as np
from io import StringIO

#parse the datetimes we get from NOAA
from datetime import datetime

# import matplotlib.pyplot as plt
from meteostat import Point, Daily
from ratelimit import limits, RateLimitException, sleep_and_retry
from concurrent.futures import ThreadPoolExecutor as PoolExecutor
# https://www1.ncdc.noaa.gov/pub/data/cdo/samples/GHCND_sample_pdf.pdf
#We find the zip code of coordinates, use that as request

#add the access token you got from NOAA
# Token = 'iRFNDKRMXHWDkBDwMJRkSHyeHmuwIaPq'
OpenWeatherKEY = 'dde707fd3b6215808ffdfee52f912d8d'
weatherapi = 'f7b866d16f5e4b7695931713221703'
test = ''

In [5]:
df = pd.read_csv('mod_CA2010Fires.csv')
df['DATETIME'] = pd.to_datetime(df['DISCOVERY_DATE'] - pd.Timestamp(0).to_julian_date(), unit='D')
print(df.head())

   Unnamed: 0  FIRE_YEAR STAT_CAUSE_DESCR   LATITUDE   LONGITUDE STATE  \
0           0       2010    Equipment Use  36.766944 -121.303056    CA   
1           1       2010    Miscellaneous  36.776944 -121.311111    CA   
2           2       2010    Miscellaneous  36.856111 -121.381111    CA   
3           3       2010    Miscellaneous  36.818056 -121.391111    CA   
4           4       2010    Miscellaneous  36.883056 -121.561944    CA   

   DISCOVERY_DATE  FIRE_SIZE        DATE      zipcode  \
0       2455335.5       70.0  2010-05-19  95075-9701,   
1       2455355.5        0.5  2010-06-08       95023,   
2       2455359.5        0.1  2010-06-12       95023,   
3       2455361.5        0.1  2010-06-14       95023,   
4       2455388.5        1.0  2010-07-11  95045-9662,   

                                             address   DATETIME  
0  Saddle Association Museum, Airline Highway, Tr... 2010-05-19  
1  Pinnacle Hills Golf Club, 7777, Airline Highwa... 2010-06-08  
2  1198, Santa

In [77]:
locations = [(32.929, -95.770), (5, 10)]

output = r""
base_url = r"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN&community=RE&longitude={longitude}&latitude={latitude}&start=20150101&end=20150305&format=JSON"

for latitude, longitude in locations:
    api_request_url = base_url.format(longitude=longitude, latitude=latitude)
    print(api_request_url)
    response = requests.get(url=api_request_url, verify=True, timeout=30.00)
    content = json.loads(response.content.decode('utf-8'))
#     print(content)
#     filename = response.headers['content-disposition'].split('filename=')[1]
#     filepath = os.path.join(output, filename)
#     with open(filepath, 'w') as file_object:
#         json.dump(content, file_object)

print(generateURL(df['LONGITUDE'][0], df['LATITUDE'][0], df['DATE'][0]))

https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN&community=RE&longitude=-95.77&latitude=32.929&start=20150101&end=20150305&format=JSON
https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN&community=RE&longitude=10&latitude=5&start=20150101&end=20150305&format=JSON
https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN&community=RE&longitude=-121.30305555&latitude=36.76694444&start=20100519&end=20100519&format=CSV


In [12]:
sz = len(df.index)
reset = []
for i in range(sz):
    lon = df['LONGITUDE'][i]
    lat = df['LATITUDE'][i]
    
    #remove '-' from string
    temp = df['DATE'][i]
    time = ''
    
    for i in temp:
        if i != '-':
            time += i
    reset.append(time)

df['DATE'] = reset
print(df.head())
    

   Unnamed: 0  FIRE_YEAR STAT_CAUSE_DESCR   LATITUDE   LONGITUDE STATE  \
0           0       2010    Equipment Use  36.766944 -121.303056    CA   
1           1       2010    Miscellaneous  36.776944 -121.311111    CA   
2           2       2010    Miscellaneous  36.856111 -121.381111    CA   
3           3       2010    Miscellaneous  36.818056 -121.391111    CA   
4           4       2010    Miscellaneous  36.883056 -121.561944    CA   

   DISCOVERY_DATE  FIRE_SIZE      DATE      zipcode  \
0       2455335.5       70.0  20100519  95075-9701,   
1       2455355.5        0.5  20100608       95023,   
2       2455359.5        0.1  20100612       95023,   
3       2455361.5        0.1  20100614       95023,   
4       2455388.5        1.0  20100711  95045-9662,   

                                             address   DATETIME  
0  Saddle Association Museum, Airline Highway, Tr... 2010-05-19  
1  Pinnacle Hills Golf Club, 7777, Airline Highwa... 2010-06-08  
2  1198, Santa Ana Road, H

In [160]:
#windspeed use 10 meters, Temperature is 2 meters
paras = ['ALLSKY_SFC_LW_DWN', 'PRECTOT', 'PS', 'QV2M', 'RH2M', 'SRF_ALB', 'T10M', 'T10M_MAX', 'T10M_MIN', 'T10M_RANGE', 'TQV', 'TS', 'WS10M', 'WS10M_MAX', 'WS10M_MIN', 'WS10M_RANGE', 'WS2M', 'WS2M_MAX', 'WS2M_MIN', 'WS2M_RANGE']
# other = [, , ]
print(len(paras))
print(paras)

p = ''
for i in paras:
    if len(p) != 0:
        p += ','
    p += i
print(p)


20
['ALLSKY_SFC_LW_DWN', 'PRECTOT', 'PS', 'QV2M', 'RH2M', 'SRF_ALB', 'T10M', 'T10M_MAX', 'T10M_MIN', 'T10M_RANGE', 'TQV', 'TS', 'WS10M', 'WS10M_MAX', 'WS10M_MIN', 'WS10M_RANGE', 'WS2M', 'WS2M_MAX', 'WS2M_MIN', 'WS2M_RANGE']
ALLSKY_SFC_LW_DWN,PRECTOT,PS,QV2M,RH2M,SRF_ALB,T10M,T10M_MAX,T10M_MIN,T10M_RANGE,TQV,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS2M,WS2M_MAX,WS2M_MIN,WS2M_RANGE


In [161]:
def generateURL(p, lon, lat, time):
    base_url = r"https://power.larc.nasa.gov/api/temporal/daily/point?parameters={parameters}&community=RE&longitude={longitude}&latitude={latitude}&start={time}&end={time}&format=CSV"
    api_request_url = base_url.format(parameters=p, longitude=lon, latitude=lat, time=time)
    return api_request_url

#Sample URL: https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN&community=RE&longitude=-121.303056&latitude=36.766944&start=20100608&end=20100608&format=JSON
print(generateURL(p, -121.303056, 36.766944, 20100608))

#Remember to cache, Loc : Date

https://power.larc.nasa.gov/api/temporal/daily/point?parameters=ALLSKY_SFC_LW_DWN,PRECTOT,PS,QV2M,RH2M,SRF_ALB,T10M,T10M_MAX,T10M_MIN,T10M_RANGE,TQV,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS2M,WS2M_MAX,WS2M_MIN,WS2M_RANGE&community=RE&longitude=-121.303056&latitude=36.766944&start=20100608&end=20100608&format=CSV


In [162]:

# example = r'https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN&community=RE&longitude=-95.77&latitude=32.929&start=20150101&end=20150101&format=CSV'
# example = generateURL(p, -121.303056, 36.766944, 20100608)
# # api_request_url = example

# # response = requests.get(url=api_request_url, verify=True, timeout=30.00)
# # content = json.loads(response.content.decode('utf-8'))
# # print(content)

# ONE_MINUTE = 60
# MAX_CALLS_PER_MINUTE = 60

# @sleep_and_retry
# @limits(calls=MAX_CALLS_PER_MINUTE, period=ONE_MINUTE)
# def access_rate_limited_api(req_url):
#     response = requests.get(url=req_url, verify=True, timeout=30.00)

#     if response.status_code != 200:
#         raise Exception('API response: {}'.format(response.status_code))
#     content = response.text
#     lines = content.split('\n')
# #     parameters = lines[len(lines)-2]
#     vals = lines[len(lines)-2]
#     return vals

# access_rate_limited_api(example)
# # YEAR,MO,DY,T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN



example = 'https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN&community=RE&longitude=-95.77&latitude=32.929&start=20150101&end=20150101&format=CSV'
# api_request_url = example

# response = requests.get(url=api_request_url, verify=True, timeout=30.00)
# content = json.loads(response.content.decode('utf-8'))
# print(content)

ONE_MINUTE = 60
MAX_CALLS_PER_MINUTE = 55

@sleep_and_retry
@limits(calls=MAX_CALLS_PER_MINUTE, period=ONE_MINUTE)
def access_rate_limited_api(req_url):
    try:
        response = requests.get(url=req_url, verify=True, timeout=None)
        if response.status_code != 200:
            raise Exception('API response: {}'.format(response.status_code))
        content = response.text
        lines = content.split('\n')
        vals = lines[len(lines)-2]
#     print(parameters)
#     print(vals)
        return vals
    except Exception:
        print('exception with:')
        print(req_url)
        SLEEPTIME.sleep(101)
        # wait = min(wait+2, 600)
        return access_rate_limited_api(req_url)

access_rate_limited_api(example)
# YEAR,MO,DY,T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN

'2010,6,8,22.77,7.52,10.39,5.13,3.91,3.0,7.14,22.0,3.61,66.38,2.45,97.68,3.67,299.49,5.45,15.07,11.61,17.72,0.0,0.14'

In [163]:
params = ['YEAR','MO','DY']
for i in paras:
    params.append(i)

mp = {}
for i in params:
    mp[i] = []
    
print(mp)

{'YEAR': [], 'MO': [], 'DY': [], 'ALLSKY_SFC_LW_DWN': [], 'PRECTOT': [], 'PS': [], 'QV2M': [], 'RH2M': [], 'SRF_ALB': [], 'T10M': [], 'T10M_MAX': [], 'T10M_MIN': [], 'T10M_RANGE': [], 'TQV': [], 'TS': [], 'WS10M': [], 'WS10M_MAX': [], 'WS10M_MIN': [], 'WS10M_RANGE': [], 'WS2M': [], 'WS2M_MAX': [], 'WS2M_MIN': [], 'WS2M_RANGE': []}


In [165]:
sz = len(df.index)
# sz = 10

# mp = {  
#     'YEAR': [],
#     'MO' : [],
#     'DY': [],
#     'T2M' :[],
#     'T2MDEW': [],
#     'T2MWET':[],
#     'TS':[],
#     'T2M_RANGE':[],
#     'T2M_MAX':[],
#     'T2M_MIN':[]
# }
#60 per minute
for i in range(sz):
    lon = df['LONGITUDE'][i]
    lat = df['LATITUDE'][i]
    time = df['DATE'][i]

    url = generateURL(p, lon, lat, time)
    s = access_rate_limited_api(url)
    vals = s.split(',')
#     if i%100 == 0:
    print('done: ' + str(i) + ' requests...')

    for i in range(len(vals)):
        mp[params[i]].append(vals[i])

print(mp)


done: 0 requests...
done: 1 requests...
done: 2 requests...
done: 3 requests...
done: 4 requests...
done: 5 requests...
done: 6 requests...
done: 7 requests...
done: 8 requests...
done: 9 requests...
{'YEAR': ['2010', '2010', '2010', '2010', '2010', '2010', '2010', '2010', '2010', '2010'], 'MO': ['5', '6', '6', '6', '7', '7', '9', '3', '5', '7'], 'DY': ['19', '8', '12', '14', '11', '29', '11', '16', '1', '3'], 'ALLSKY_SFC_LW_DWN': ['21.05', '22.77', '16.59', '18.62', '28.95', '24.01', '17.03', '12.72', '13.65', '15.52'], 'PRECTOT': ['5.88', '7.52', '2.42', '3.27', '6.0', '21.42', '1.4', '9.81', '2.94', '0.88'], 'PS': ['7.82', '10.39', '97.42', '97.6', '16.87', '6.41', '97.54', '6.71', '97.58', '98.55'], 'QV2M': ['3.7', '5.13', '1.22', '1.75', '3.2', '314.62', '0.16', '296.16', '1.73', '0.34'], 'RH2M': ['4.12', '3.91', '2.59', '3.47', '5.9', '10.81', '4.22', '12.37', '3.38', '3.28'], 'SRF_ALB': ['3.26', '3.0', '294.6', '304.59', '4.13', '2.78', '316.75', '1.02', '282.08', '295.09'], 'T1

In [167]:
print(len(df.index))

31858


In [133]:
#http://rmirror.lau.edu.lb/web/packages/nasapower/nasapower.pdf
#list of params and their short names